In [ ]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import time
import csv
import datetime
from selenium import webdriver
import datetime
import random
import os

In [ ]:
# Configurations
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

driver = webdriver.Chrome(
    r'//Users/kunaldeshpande/Documents/chromedriver 2')

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
           "X-Amzn-Trace-Id": "Root=1-620b953d-2201315c6de7caf52ed38203", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
           "Accept-Encoding": "gzip, deflate, br"}

column_header = ['Title', 'Price', 'Date', 'URL']



In [ ]:
def readB2BSiteLinks(webSiteType):
    webSiteLink = ""
    switch = {
        "Amazon": r"/Users/kunaldeshpande/Desktop/project/Scrap/amazon/amazonlinks.csv"
    }
    df = pd.read_csv(switch[webSiteType])
    return df

In [ ]:
def scrapB2BSite(siteLinks):
    page = requests.get(siteLinks, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    formattedSoup = BeautifulSoup(soup.prettify(), "html.parser")
    try:
        title = formattedSoup.find(id='productTitle').get_text()
        price_parent = formattedSoup.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
        price = price.strip()[1:]
        title = title.strip()
        today = datetime.datetime.now().isoformat()
        data = [title, price, today, siteLinks]
        return data
    except AttributeError as e:
        print(e)
    except Exception as e:
        print(e)
    return []

In [ ]:
def writeB2BSiteDataToCsvFile(file_path, data, column_header):
    has_header = False
    # checking if the csv file contains headers already
    if(os.path.isfile(file_path)):
        with open(file_path, 'r') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)
            # extracting field names through first row
            fields = next(csvreader)
            if(len(fields)>0):
                has_header = True
            csvfile.close()
      
    with open(file_path, 'a', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        if(not has_header):
            writer.writerow(column_header)
            has_header = True
        writer.writerow(data)

In [ ]:
# reading the B2B site links
b2b_site_links = readB2BSiteLinks("Amazon")

In [ ]:
print(b2b_site_links)

In [ ]:
for site_links in b2b_site_links['Link']:
    #starting the scrapping process
    data = scrapB2BSite(site_links)
    print(data)
    sleepTimeInSeconds = random.randrange(2,4)
    time.sleep(sleepTimeInSeconds)
    # writing the processed output to a csv file
    if(len(data)>0):
        writeB2BSiteDataToCsvFile('/Users/kunaldeshpande/Desktop/project/Scrap/29092022.csv', data, column_header)